In [ ]:
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
import time
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
with open('../../data/processed/df_selected_05.pkl', 'rb') as f:
    df = pickle.load(f)

# X, y 분리
y = df['SalePrice']
X = df.drop('SalePrice', axis=1)

print(f"데이터 형태: X {X.shape}, y {y.shape}")
print(f"특성 수: {X.shape[1]}")

# 교차 검증 설정
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


데이터 형태: X (1460, 15), y (1460,)
특성 수: 15


In [2]:
# 기본 모델 성능 측정
base_model = RandomForestRegressor(random_state=42)
base_scores = cross_val_score(base_model, X, y, cv=kfold, scoring='r2')

print("=== 기본 모델 성능 ===")
print(f"R² 평균: {base_scores.mean():.4f} (±{base_scores.std():.4f})")
print(f"개별 R² 점수: {base_scores}")

baseline_r2 = base_scores.mean()


=== 기본 모델 성능 ===
R² 평균: 0.8335 (±0.0475)
개별 R² 점수: [0.85634433 0.86162314 0.73870447 0.85165434 0.85910691]


In [3]:
print("=== 1단계: 기본 파라미터 튜닝 ===")
print("n_estimators와 max_depth 최적화 중...")

# 1단계 파라미터 그리드
param_grid_1 = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 15, 20, 25, 30]
}

rf_1 = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_1 = GridSearchCV(
    rf_1, param_grid_1, 
    cv=kfold, 
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

start_time = time.time()
grid_1.fit(X, y)
end_time = time.time()

print(f"\n1단계 완료 시간: {end_time - start_time:.2f}초")
print(f"최적 파라미터: {grid_1.best_params_}")
print(f"최적 R² 점수: {grid_1.best_score_:.4f}")
print(f"성능 향상: {grid_1.best_score_ - baseline_r2:.4f}")

# 1단계 최적 파라미터 저장
best_params_1 = grid_1.best_params_.copy()


=== 1단계: 기본 파라미터 튜닝 ===
n_estimators와 max_depth 최적화 중...
Fitting 5 folds for each of 24 candidates, totalling 120 fits

1단계 완료 시간: 37.92초
최적 파라미터: {'max_depth': 10, 'n_estimators': 300}
최적 R² 점수: 0.8365
성능 향상: 0.0030


In [4]:
print("\n=== 2단계: 과적합 제어 파라미터 튜닝 ===")
print("min_samples_split, min_samples_leaf, max_features 최적화 중...")

# 2단계 파라미터 그리드 (1단계 최적값 + 새로운 파라미터들)
param_grid_2 = {
    'n_estimators': [best_params_1['n_estimators']],
    'max_depth': [best_params_1['max_depth']],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', 0.5, 0.7, 1.0]
}

rf_2 = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_2 = GridSearchCV(
    rf_2, param_grid_2,
    cv=kfold,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

start_time = time.time()
grid_2.fit(X, y)
end_time = time.time()

print(f"\n2단계 완료 시간: {end_time - start_time:.2f}초")
print(f"최적 파라미터: {grid_2.best_params_}")
print(f"최적 R² 점수: {grid_2.best_score_:.4f}")
print(f"1단계 대비 향상: {grid_2.best_score_ - grid_1.best_score_:.4f}")
print(f"기준선 대비 향상: {grid_2.best_score_ - baseline_r2:.4f}")

# 2단계 최적 파라미터 저장
best_params_2 = grid_2.best_params_.copy()



=== 2단계: 과적합 제어 파라미터 튜닝 ===
min_samples_split, min_samples_leaf, max_features 최적화 중...
Fitting 5 folds for each of 80 candidates, totalling 400 fits

2단계 완료 시간: 61.50초
최적 파라미터: {'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
최적 R² 점수: 0.8427
1단계 대비 향상: 0.0062
기준선 대비 향상: 0.0092


In [5]:
print("\n=== 3단계: 세부 조정 ===")
print("max_samples 최적화 중...")

# 3단계 파라미터 그리드
param_grid_3 = {
    'n_estimators': [best_params_2['n_estimators']],
    'max_depth': [best_params_2['max_depth']],
    'min_samples_split': [best_params_2['min_samples_split']],
    'min_samples_leaf': [best_params_2['min_samples_leaf']],
    'max_features': [best_params_2['max_features']],
    'max_samples': [0.7, 0.8, 0.9, 1.0]
}

rf_3 = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_3 = GridSearchCV(
    rf_3, param_grid_3,
    cv=kfold,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

start_time = time.time()
grid_3.fit(X, y)
end_time = time.time()

print(f"\n3단계 완료 시간: {end_time - start_time:.2f}초")
print(f"최적 파라미터: {grid_3.best_params_}")
print(f"최적 R² 점수: {grid_3.best_score_:.4f}")
print(f"2단계 대비 향상: {grid_3.best_score_ - grid_2.best_score_:.4f}")
print(f"기준선 대비 향상: {grid_3.best_score_ - baseline_r2:.4f}")

# 최종 최적 파라미터
final_best_params = grid_3.best_params_.copy()



=== 3단계: 세부 조정 ===
max_samples 최적화 중...
Fitting 5 folds for each of 4 candidates, totalling 20 fits

3단계 완료 시간: 2.52초
최적 파라미터: {'max_depth': 10, 'max_features': 0.5, 'max_samples': 0.7, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
최적 R² 점수: 0.8428
2단계 대비 향상: 0.0001
기준선 대비 향상: 0.0093


In [6]:
print("\n=== 최종 결과 비교 ===")

# 최종 최적 모델로 상세 검증
final_model = RandomForestRegressor(**final_best_params, random_state=42)
final_r2_scores = cross_val_score(final_model, X, y, cv=kfold, scoring='r2')
final_mse_scores = cross_val_score(final_model, X, y, cv=kfold, scoring='neg_mean_squared_error')
final_rmse_scores = np.sqrt(-final_mse_scores)

print(f"기준선 (기본 모델):")
print(f"  R² 평균: {baseline_r2:.4f}")

print(f"\n최종 최적화 모델:")
print(f"  R² 평균: {final_r2_scores.mean():.4f} (±{final_r2_scores.std():.4f})")
print(f"  RMSE 평균: {final_rmse_scores.mean():.4f} (±{final_rmse_scores.std():.4f})")
print(f"  개별 R² 점수: {final_r2_scores}")

print(f"\n성능 향상:")
print(f"  R² 향상: {final_r2_scores.mean() - baseline_r2:.4f}")
print(f"  향상률: {((final_r2_scores.mean() - baseline_r2) / baseline_r2 * 100):.2f}%")

print(f"\n최종 최적 하이퍼파라미터:")
for param, value in final_best_params.items():
    print(f"  {param}: {value}")



=== 최종 결과 비교 ===
기준선 (기본 모델):
  R² 평균: 0.8335

최종 최적화 모델:
  R² 평균: 0.8428 (±0.0370)
  RMSE 평균: 0.1566 (±0.0168)
  개별 R² 점수: [0.85081276 0.86895611 0.77032788 0.85421318 0.86971823]

성능 향상:
  R² 향상: 0.0093
  향상률: 1.12%

최종 최적 하이퍼파라미터:
  max_depth: 10
  max_features: 0.5
  max_samples: 0.7
  min_samples_leaf: 4
  min_samples_split: 2
  n_estimators: 300


In [7]:
# 최적 모델로 특성 중요도 분석
final_model.fit(X, y)
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== 특성 중요도 (Top 10) ===")
print(feature_importance.head(10).to_string(index=False))

# 최적 모델 저장
with open('../../models/optimized_random_forest_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)
    
print("\n최적화된 모델이 '../../models/optimized_random_forest_model.pkl'로 저장되었습니다.")



=== 특성 중요도 (Top 10) ===
       feature  importance
Quality_x_Area    0.374679
   OverallQual    0.189914
       TotalSF    0.188236
     GrLivArea    0.040941
  EffectiveAge    0.036731
   TotalBsmtSF    0.034011
    GarageCars    0.030231
    GarageArea    0.028985
AboveGradeBath    0.023110
      1stFlrSF    0.019184

최적화된 모델이 'optimized_random_forest_model.pkl'로 저장되었습니다.
